# Análisis del Filtro de Kalman
Este notebook implementa paso a paso el filtro de Kalman, incluyendo estimación de estado, predicción, actualización con medición y análisis de convergencia.

## 1. Importar librerías necesarias
Importaremos NumPy para cálculos numéricos y Matplotlib para visualización.

In [2]:
!pip install numpy pandas matplotlib pykalman

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/5.1 MB ? eta -:--:--Downloading numpy-2.3.4-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 1.4 MB/s  0:00:03 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 1.4 MB/s  0:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/10.7 MB ? eta -:--:--Downloading pandas-2.3.3-cp313-cp313-macosx_11_0_arm64.whl (10.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 1.8 MB/s  0:00:05eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 1.8 MB/s  0:00:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/8.1 MB ? eta -:--:--Downloading matplotlib-3.10.7-cp313-cp313-macosx_11_0_arm64.whl (8.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 1.4 MB/s  0:00:05 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB

In [3]:
import numpy as np
import matplotlib.pyplot as plt

# Configuración para gráficos en notebook
%matplotlib inline

## 2. Inicializar el vector de estado y la matriz de covarianza
Definimos los valores iniciales para el estado y la incertidumbre.

In [4]:
# Estado inicial (ejemplo: posición y velocidad)
x = np.array([[0],    # posición inicial
              [1]])   # velocidad inicial

# Matriz de covarianza inicial (incertidumbre)
P = np.array([[1, 0],
              [0, 1]])

print('Estado inicial x:\n', x)
print('Matriz de covarianza inicial P:\n', P)

Estado inicial x:
 [[0]
 [1]]
Matriz de covarianza inicial P:
 [[1 0]
 [0 1]]


## 3. Predecir el estado y la covarianza para el siguiente paso de tiempo
Utilizamos la matriz de transición de estado para predecir el siguiente estado y la nueva covarianza.

In [5]:
# Matriz de transición de estado (ejemplo: modelo de movimiento constante)
F = np.array([[1, 1],
              [0, 1]])

# Varianza del modelo (ruido del proceso)
Q = np.array([[0.1, 0],
              [0, 0.1]])

# Predicción del estado y la covarianza
x_pred = F @ x
P_pred = F @ P @ F.T + Q

print('Estado predicho x_pred:\n', x_pred)
print('Covarianza predicha P_pred:\n', P_pred)

Estado predicho x_pred:
 [[1]
 [1]]
Covarianza predicha P_pred:
 [[2.1 1. ]
 [1.  1.1]]


## 4. Calcular la ganancia de Kalman
La ganancia de Kalman determina cuánto se ajusta la predicción con la medición observada.

In [6]:
# Matriz de observación (solo medimos la posición)
H = np.array([[1, 0]])

# Varianza de la medición (ruido de medición)
R = np.array([[0.5]])

# Calcular la ganancia de Kalman
S = H @ P_pred @ H.T + R
K = P_pred @ H.T @ np.linalg.inv(S)

print('Ganancia de Kalman K:\n', K)

Ganancia de Kalman K:
 [[0.80769231]
 [0.38461538]]


## 5. Realizar una medición y actualizar las estimaciones
Utilizamos una medición simulada para actualizar el estado y la covarianza.

In [8]:
# Simulamos una medición (por ejemplo, posición observada)
z = np.array([[1.2]])

# Actualización del estado y la covarianza
x_upd = x_pred + K @ (z - H @ x_pred)
P_upd = (np.eye(2) - K @ H) @ P_pred

print('Estado actualizado x_upd:\n', x_upd)
print('Covarianza actualizada P_upd:\n', P_upd)

Estado actualizado x_upd:
 [[1.16153846]
 [1.07692308]]
Covarianza actualizada P_upd:
 [[0.40384615 0.19230769]
 [0.19230769 0.71538462]]


## 6. Elegir una varianza del modelo mayor que la de la medición
Ahora aumentamos la varianza del modelo y repetimos el análisis.

In [9]:
# Nueva varianza del modelo (mayor que la de medición)
Q_new = np.array([[2.0, 0],
                  [0, 2.0]])

# Repetimos el ciclo de predicción y actualización
x_pred2 = F @ x_upd
P_pred2 = F @ P_upd @ F.T + Q_new
S2 = H @ P_pred2 @ H.T + R
K2 = P_pred2 @ H.T @ np.linalg.inv(S2)
z2 = np.array([[1.5]])  # nueva medición simulada
x_upd2 = x_pred2 + K2 @ (z2 - H @ x_pred2)
P_upd2 = (np.eye(2) - K2 @ H) @ P_pred2

print('Estado actualizado con mayor varianza x_upd2:\n', x_upd2)
print('Covarianza actualizada con mayor varianza P_upd2:\n', P_upd2)

Estado actualizado con mayor varianza x_upd2:
 [[1.59221902]
 [0.90951009]]
Covarianza actualizada con mayor varianza P_upd2:
 [[0.43756004 0.11335255]
 [0.11335255 2.50960615]]


## 7. ¿Converge la estimación del Filtro de Kalman?
Analizaremos la evolución de la covarianza y el estado para ver si el filtro converge.

In [ ]:
# Simulación de varios pasos para analizar convergencia
n_steps = 20
x_hist = []
P_hist = []
z_hist = []

x_k = x.copy()
P_k = P.copy()
Q_sim = Q_new.copy()

for k in range(n_steps):
    # Predicción
    x_pred_k = F @ x_k
    P_pred_k = F @ P_k @ F.T + Q_sim
    # Medición simulada (ruido normal)
    z_k = np.array([[1.0 + 0.5 * np.random.randn()]])
    # Ganancia de Kalman
    S_k = H @ P_pred_k @ H.T + R
    K_k = P_pred_k @ H.T @ np.linalg.inv(S_k)
    # Actualización
    x_k = x_pred_k + K_k @ (z_k - H @ x_pred_k)
    P_k = (np.eye(2) - K_k @ H) @ P_pred_k
    # Guardar historia
    x_hist.append(x_k.flatten())
    P_hist.append(np.diag(P_k))
    z_hist.append(z_k.flatten())

x_hist = np.array(x_hist)
P_hist = np.array(P_hist)
z_hist = np.array(z_hist)

## 8. Visualización de resultados y análisis de convergencia
Graficamos la evolución de las estimaciones y la covarianza para observar el comportamiento del filtro.

In [ ]:
# Graficar estimación de posición y mediciones
plt.figure(figsize=(10,5))
plt.plot(x_hist[:,0], label='Estimación de posición (KF)')
plt.plot(z_hist, 'o', label='Mediciones')
plt.xlabel('Paso de tiempo')
plt.ylabel('Posición')
plt.title('Estimación de posición vs mediciones')
plt.legend()
plt.show()

# Graficar evolución de la covarianza
plt.figure(figsize=(10,5))
plt.plot(P_hist[:,0], label='Covarianza de posición')
plt.plot(P_hist[:,1], label='Covarianza de velocidad')
plt.xlabel('Paso de tiempo')
plt.ylabel('Valor de covarianza')
plt.title('Evolución de la covarianza')
plt.legend()
plt.show()

# Explicación sobre la convergencia
print('''\nExplicación:
El filtro de Kalman ajusta sus estimaciones en cada paso, combinando la predicción del modelo y la medición observada. Cuando la varianza del modelo es mayor que la de la medición, el filtro confía más en las mediciones y menos en el modelo. La covarianza tiende a estabilizarse, mostrando que el filtro converge y las estimaciones se vuelven más precisas con el tiempo, aunque dependen de la relación entre las varianzas del modelo y la medición.''')

In [ ]:
import pandas as pd

df = pd.read_csv('Appendix 1 - xy_motion_kalman_filter_example.csv')
df.head()